In [1]:
import os; os.getcwd()

'/Users/jessicahong/NLP_study/04_llm_related/applications'

In [3]:
import sys
import os

def set_project_root(levels_up=2):
    """
    Add the project root directory (levels_up above current file) to sys.path
    so Python can find modules/packages there.
    """
    current_path = os.path.abspath(os.path.dirname(__file__))
    project_root = current_path
    for _ in range(levels_up):
        project_root = os.path.dirname(project_root)

    if project_root not in sys.path:
        sys.path.insert(0, project_root)
        print(f"Project root set to ( {levels_up} levels up): {project_root}")


In [4]:
import pandas as pd

# movies.json 파일 읽기
movies = pd.read_json('../../12_data/movies.json')
print("movies 데이터 미리보기:")
print(movies.head())
print("movies shape:", movies.shape)

# ratings.json 파일 읽기
ratings = pd.read_json('../../12_data/ratings.json')
print("\nratings 데이터 미리보기:")
print(ratings.head())
print("ratings shape:", ratings.shape)

movies 데이터 미리보기:
   movieId                                        title
0       31                       Dangerous Minds (1995)
1       32    Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
2       39                              Clueless (1995)
3       80  White Balloon, The (Badkonake sefid) (1995)
4      177                     Lord of Illusions (1995)
movies shape: (49, 2)

ratings 데이터 미리보기:
   userId  movieId  rating
0     156     3167     3.0
1     514     1258     4.0
2      32       39     3.0
3     305    55276     5.0
4     603     2020     4.0
ratings shape: (50, 3)


In [5]:
ratings_small = ratings.sample(n=50, random_state=42)
print("\n샘플링된 ratings_small:")
print(ratings_small.head())
print(ratings_small.shape)


샘플링된 ratings_small:
    userId  movieId  rating
13     116      356     4.0
39      80     2513     4.0
30     183     1408     4.0
45     438     6503     0.5
17     409     3868     4.0
(50, 3)


In [4]:
print(movies.head())
print(ratings.head())

   movieId                                        title
0       31                       Dangerous Minds (1995)
1       32    Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
2       39                              Clueless (1995)
3       80  White Balloon, The (Badkonake sefid) (1995)
4      177                     Lord of Illusions (1995)
   userId  movieId  rating
0     156     3167     3.0
1     514     1258     4.0
2      32       39     3.0
3     305    55276     5.0
4     603     2020     4.0


In [5]:
user_item_matrix = ratings_small.pivot_table(
    index='userId',
    columns='movieId',
    values='rating',
    aggfunc='first',      # 중복 없으면 mean 대신 first
    fill_value=0          # 결측치 0으로 대체
)
print(user_item_matrix.head())

movieId  31      32      39      80      177     204     356     474     \
userId                                                                    
6           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
16          0.0     0.0     0.0     0.0     0.0     2.0     0.0     0.0   
32          0.0     0.0     3.0     0.0     0.0     0.0     0.0     0.0   
42          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
45          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  541     587     ...  26736   42761   48394   50872   51662   55276   \
userId                   ...                                                   
6           0.0     5.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
16          0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
32          0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
42          0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0

In [6]:
#create a user-item rating matrix
user_item_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')
# pivot table=crete a matrix with userId as rows (index) movieid as columns and rating as values

print(user_item_matrix.head())

movieId  31      32      39      80      177     204     356     474     \
userId                                                                    
6           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
16          NaN     NaN     NaN     NaN     NaN     2.0     NaN     NaN   
32          NaN     NaN     3.0     NaN     NaN     NaN     NaN     NaN   
42          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
45          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  541     587     ...  26736   42761   48394   50872   51662   55276   \
userId                   ...                                                   
6           NaN     5.0  ...     NaN     NaN     NaN     NaN     NaN     NaN   
16          NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
32          NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
42          NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     N

In [7]:
top_movies = ratings['movieId'].value_counts().head(10)
print(movies[movies['movieId'].isin(top_movies.index)])

    movieId                                             title
7       474                        In the Line of Fire (1993)
15     1203                               12 Angry Men (1957)
18     1408                  Last of the Mohicans, The (1992)
23     2355                              Bug's Life, A (1998)
27     3167                           Carnal Knowledge (1971)
31     4816                                  Zoolander (2001)
32     5065                    Mothman Prophecies, The (2002)
41    48394  Pan's Labyrinth (Laberinto del fauno, El) (2006)
46   111362                 X-Men: Days of Future Past (2014)
48   119155    Night at the Museum: Secret of the Tomb (2014)


In [8]:
#imports the consine_similarity func from sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
#user_item_matrix.fillna(0) -> replaces all missing values (NaN) in the user-tim rating matrix with 0
user_item_filled = user_item_matrix.fillna(0)

#computes the consine similarity btw every pair of users based on their rating vectors
user_sim = cosine_similarity(user_item_filled)
print('User similarity matrix shape:', user_sim.shape)

User similarity matrix shape: (48, 48)


In [ ]:
#install surprize library 
!pip install scikit-surprise
!conda install -c conda-forge scikit-surprise


#import required modules
from surprise import Dataset, Reader, SVD


#import the cross-validation func for model evaluation
from surprise.model_selection import cross_validate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Channels:
 - conda-forge
 - anaconda
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libsqlite-3.46.0           |       hfb93653_0         811 KB  conda-forge
    libzlib-1.2.13             |       hfb2fe0b_6          46 KB  conda-forge
    python-3.12.2              |hdf0ec26_0_cpython        12.5 MB  conda-forge
    python_abi-3.12            |          7_cp312           7 KB  conda-forge
    scikit-surprise-1.1.4      |  py312h911a38c_1         332 KB  conda-forge
    zlib-1.2.13                |       hfb2fe0b_6          76 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        13.7 MB

The following NEW packages will be INSTALLED:

  li

In [ ]:
#install surprize library 
!pip install scikit-surprise
!conda install -c conda-forge scikit-surprise


#import required modules
from surprise import Dataset, Reader, SVD


#import the cross-validation func for model evaluation
from surprise.model_selection import cross_validate

In [1]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm  # For progress bar (install with pip install tqdm if needed)

MODEL_NAME = "gpt2"
PROMPT_FILE = "./12_data/movielens_prompt_response.json"

# Load prompt-response data from JSON file
with open(PROMPT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

results = []
errors = []

# Iterate through all prompts with a progress bar
for item in tqdm(data, desc="Processing prompts"):
    try:
        prompt = item['prompt']
        # Tokenize the prompt and prepare input tensor
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
        # Generate model response
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            num_return_sequences=1
        )
        # Decode the generated response
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Store prompt, reference response, and model response
        results.append({
            "prompt": prompt,
            "reference_response": item['response'],
            "gpt2_response": generated
        })
    except Exception as e:
        print(f"\n[Error] prompt: {item['prompt']}\n{e}")
        errors.append({"prompt": item['prompt'], "error": str(e)})

# Save results to a JSON file
with open("gpt2_generated_responses.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

# Save errors to a separate file if any occurred
if errors:
    with open("gpt2_generation_errors.json", "w", encoding="utf-8") as f:
        json.dump(errors, f, ensure_ascii=False, indent=2)
    print(f"\n[Notice] Errors occurred for {len(errors)} prompts, saved to gpt2_generation_errors.json.")

# Print a sample result (optional)
if results:
    print("\n[Sample Result] First item:")
    print("Prompt:", results[0]['prompt'])
    print("Reference response:", results[0]['reference_response'])
    print("GPT-2 generated response:", results[0]['gpt2_response'])
else:
    print("\n[Notice] No results were generated.")



Processing prompts:  10%|██▍                     | 1/10 [00:01<00:12,  1.43s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Processing prompts: 100%|███████████████████████| 10/10 [00:10<00:00,  1.07s/it]


[Sample Result] First item:
Prompt: Can you recommend a good comedy movie?
Reference response: You might enjoy 'The Big Lebowski'. It's a classic comedy with a unique sense of humor.
GPT-2 generated response: Can you recommend a good comedy movie? What else do you think would be interesting to watch? Leave your suggestions in the comments below.


In [ ]:
#----------------gpt-j-6B---------------------------

In [ ]:
#!pip install "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2" "accelerate>=0.16.0,<1"

In [ ]:
#model_name = "databricks/dolly-v2-3b"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# Prepare the prompt for the language model
#prompt = "Recommend a good thriller movie."

# Tokenize the prompt and convert it to PyTorch tensors, with truncation
#inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)

# Generate a response from the model with a maximum length of 100 tokens
#outputs = model.generate(**inputs, max_length=100)

# Decode the generated tokens into a readable string and print the result
#print(tokenizer.decode(outputs[0], skip_special_tokens=True))
